In [1]:
#pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic     # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, detection):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR to RGB
        image.flags.writeable = False                   # Ustawienie obrazu jako niezapisywalnego
        results = detection.process(image)              # Przeprowadzenie detekcji
        image.flags.writeable = True                    # Ustawienie obrazu jako zapisywalnego
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # RGB to BGR
    except Exception as e:
        print(f"Error during mediapipe detection: {e}")
        return None, None
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    
    # Poza
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(45, 95, 44), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(98, 188, 151), thickness=2, circle_radius=2)
                             ) 
    # Lewa dłoń 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(73, 37, 245), thickness=3, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(117, 103, 250), thickness=3, circle_radius=2)
                             ) 
    # Prawa dłoń   
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(165, 93, 113), thickness=3, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(229, 197, 211), thickness=3, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [10]:
# Ścieżka do zapisywania danych
DATA_PATH = os.path.join('MP_Data')

# Lista akcji do rozpoznawania
actions = np.array(['car', 'coffee', 'face', 'fall', 'friday', 'gold', 'goodbye', 'heart', 'hello', 'iloveyou', 'key', 'monday', 'music', 'phone', 
                    'pretty', 'saturday', 'spring', 'summer', 'sunday', 'tea', 'thanks', 'thursday', 'tuesday', 'wednesday', 'winter'])

# Licznik wideo i klatek
videos = 30                                             
frames = 30  

# Folder startowy                                           
start_folder = 120              

In [10]:
def createFolders():
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if os.path.exists(action_path) and os.path.isdir(action_path) and os.listdir(action_path):
            dirmax = np.max(np.array(os.listdir(action_path)).astype(int))
        else:
            dirmax = -1
        for sequence in range(videos):
            try: 
                os.makedirs(os.path.join(DATA_PATH, action, str(dirmax + sequence + 1)))
            except Exception as e:
                print(f"Error creating folder: {e}")

# createFolders()


In [36]:
cap = cv2.VideoCapture(0)

# Zbieranie danych
def collect_data():
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        for sequence in range(start_folder, start_folder + videos):                         # Pętla przez video
            for frame_num in range(frames):                                                 # Pętla przez frames

                ret, frame = cap.read()                                                     # Przechwytywanie wideo
                image, results = mediapipe_detection(frame, holistic)                       # Detekcja

                draw_styled_landmarks(image, results)                                       # Rysowanie punktów orientacyjnych
                
                # Wyświetlanie teksu
                cv2.rectangle(image, (0, 0), (200, 25), (0, 0, 0), -1)
                cv2.putText(image, '{} nr: {}'.format(action, sequence), (5, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                if frame_num == 0: 
                    cv2.putText(image, 'NEXT', (200, 220), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 4, cv2.LINE_AA)
                    cv2.imshow('Data collector', image)
                    cv2.waitKey(700)
                else: 
                    cv2.imshow('Data collector', image)
                
                # Zapis punktów kluczowych
                keypoints = extract_keypoints(results)                                      # Wyodrębnianie punktów
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))   # Ścieżka
                try:
                    np.save(npy_path, keypoints)                                            # Zapis
                except Exception as e:
                    print(f"Error saving keypoints: {e}")                                               
                

                # Przerwanie 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    return
                        
        cap.release()
        cv2.destroyAllWindows()
        
# 'car', 'coffee', 'face', 'fall', 'friday', 'gold', 'goodbye', 'heart', 'hello', 'iloveyou', 'key', 'monday', 'music', 
# 'phone', 'pretty', 'saturday', 'spring', 'summer', 'sunday', 'tea', 'thanks', 'thursday', 'tuesday', 'wednesday', 'winter'
action = "summer"    # current action 
collect_data()


In [82]:
cap.release()
cv2.destroyAllWindows()